In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import random
import numpy as np
import networkx as nx
import scipy.sparse as sp
import pandas as pd 
import pysnooper
from tensorflow.python import debug as tf_debug
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.utils import plot_model

In [2]:
config = tf.ConfigProto() # 定义TensorFlow配置
config.gpu_options.allow_growth = True # 配置GPU动态分配，按需增长 
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
K.set_session(sess)  # set this TensorFlow session as the default session for Keras
K.set_session(tf_debug.LocalCLIDebugWrapperSession(sess))

## 加载数据

In [3]:
idx_features_labels = np.genfromtxt("./data/cora.content", dtype=np.dtype(str))

In [4]:
idx_features_labels.shape

(2708, 1435)

In [5]:
idx_features_labels[0]  # 即第一个是所属节点，中间部分是节点特征 最后一个是类别标签

array(['31336', '0', '0', ..., '0', '0', 'Neural_Networks'], dtype='<U22')

In [6]:
# 未进行稀疏性压缩
features = np.array(idx_features_labels[:, 1:-1], dtype=np.float32)

In [7]:
features.shape  # 节点特征

(2708, 1433)

### 提取标签，并转化为数组

In [8]:
# 提取样本的标签，并将其转换为one-hot编码形式
def encode_label(labels):
    classes = set(labels)
    # 生成class对应的向量的字典
    classes_dict = {c: np.eye(len(classes))[index, :] for index, c in enumerate(classes)}
    # 生成向量
    label_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return label_onehot, classes_dict
labels_list = idx_features_labels[:, -1]
labels, labels_class = encode_label(labels_list)

In [9]:
labels.shape, labels_class

((2708, 7),
 {'Case_Based': array([0., 0., 1., 0., 0., 0., 0.]),
  'Genetic_Algorithms': array([0., 0., 0., 1., 0., 0., 0.]),
  'Neural_Networks': array([0., 0., 0., 0., 0., 1., 0.]),
  'Probabilistic_Methods': array([0., 0., 0., 0., 0., 0., 1.]),
  'Reinforcement_Learning': array([0., 0., 0., 0., 1., 0., 0.]),
  'Rule_Learning': array([0., 1., 0., 0., 0., 0., 0.]),
  'Theory': array([1., 0., 0., 0., 0., 0., 0.])})

### 获取节点，生成节点和索引的字典表

In [10]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
print('\n节点数:',len(idx))


节点数: 2708


In [11]:
# 生成节点和索引的字典
idx_map = {node: index for index, node in enumerate(idx)}

In [12]:
i = 0 
for node, id_ in idx_map.items():
    i = i + 1
    if i > 5:
        break 
    print(node,":", id_)

851968 : 1211
1155073 : 1686
249858 : 20
77829 : 825
102406 : 1181


### 读取节点之间的关系

In [13]:
# 读取node之间的关系
edges_unordered = np.genfromtxt("./data/cora.cites", dtype=np.int32)

In [14]:
edges_unordered.shape

(5429, 2)

In [15]:
# 将节点映射到对应的索引
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)

In [16]:
edges.shape # 变的条数

(5429, 2)

In [17]:
edges[:10]

array([[ 163,  402],
       [ 163,  659],
       [ 163, 1696],
       [ 163, 2295],
       [ 163, 1274],
       [ 163, 1286],
       [ 163, 1544],
       [ 163, 2600],
       [ 163, 2363],
       [ 163, 1905]], dtype=int32)

### 构建邻接矩阵

In [18]:
adj = np.zeros((labels.shape[0], labels.shape[0]))
adj.shape

(2708, 2708)

In [19]:
for edge in edges:
    adj[edge[0], edge[1]] = 1.

In [20]:
adj

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
# 生成对称矩阵
adj = adj + adj.T  - np.diagflat(np.diag(adj))

In [22]:
print('Dataset has {} nodes, {} edges, {} features.'.format(adj.shape[0], edges.shape[0], features.shape[1]))

Dataset has 2708 nodes, 5429 edges, 1433 features.


In [23]:
X = features
A = adj
y = labels

In [24]:
X.shape, A.shape, y.shape

((2708, 1433), (2708, 2708), (2708, 7))

In [25]:
# 划分训练集和测试集
# 划分label
def get_splits(y):
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    y_train = np.zeros(y.shape, dtype=np.int32)
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train]
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]
    
    mask = np.zeros(y.shape[0])  # 2708
    mask[idx_train] = 1
    # 训练数据的样本掩码
    train_mask = np.array(mask, dtype=np.bool)
    
    return y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask

In [26]:
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In [27]:
y_train.shape, sum(train_mask)

((2708, 7), 140)

### 对特征进行归一化

In [28]:
# 对特征进行归一化
X = X / np.sum(X, axis=1).reshape((-1, 1))

### 对邻接矩阵的处理

#### filter为局部池化/切比雪夫

In [29]:
def normalize_adj(adj, symmetric=True):
    """
    对邻接矩阵进行归一化处理
    :param adj: 邻接矩阵(密集矩阵)
    :param symmetric: 是否对称
    :return: 归一化后的邻接矩阵
    """
    # 如果邻接矩阵为对称矩阵，得到对称归一化邻接矩阵
    # D^(-1/2) * A * D^(-1/2)
    if symmetric:
        # 得到度矩阵
        D = np.sum(adj, axis=1)
        # 然后取根号
        D = np.power(D, -1/2) 
        # 生成对角矩阵
        D = np.diagflat(D.flatten(), 0) # 主对角线
        print("对角矩阵:\n", D)
        #  D^(-1/2) * A * D^(-1/2)
        a_norm = D * adj * D
    else:
        # 如果邻接矩阵不是对称矩阵，得到随机游走正则化拉普拉斯算子
        # D^(-1) * A
        D = np.diagflat(np.power(np.sum(adj, axis=1), -1).flatten(), 0) # 主对角线
        a_norm = D * adj
    return a_norm

In [30]:
# 过滤器
FILTER = 'chebyshev'  # 'chebyshev'两种滤波器
# 是否对称正则化
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
# 多项式的最大阶数
MAX_DEGREE = 2  # maximum polynomial degree

# 当过滤器为局部池化过滤器时
if FILTER == "localpool":
    # 加入自连接的邻接矩阵, 在邻接矩阵中加入自连接(因为自身信息很重要)
    A = A + np.eye(A.shape[0])
    # 对加入自连接的邻接矩阵进行对称归一化处理
    A_norm = normalize_adj(A, symmetric=SYM_NORM) # 对应传播规则
    support = 1 # 只考虑一阶情况
    
    # 特征矩阵和邻接矩阵
    graph = [X, A_norm]
#     G = [keras.layers.Input(shape=(None,))]
#     G = [keras.layers.Input(shape=(None, None),)]
    
    
# 当过滤器为切比雪夫多项式时
elif FILTER == "chebyshev":
    # 对拉普拉斯矩阵进行归一化处理，得到对称规范化的拉普拉斯矩阵
    # 对称归一化的邻接矩阵
    A_norm = normalize_adj(A, symmetric=True)
    # 得到图拉普拉斯矩阵，L = I - D ^ (-1/2) * A * D ^ (-1/2)
    laplacian = np.eye(A_norm.shape[0]) - A_norm
    # 重新调整对称归一化的图拉普拉斯矩阵，得到其简化版本
    try:
        # 计算对称归一化图拉普拉斯矩阵的最大特征值
        largest_eigenvalues = sp.linalg.eigsh(laplacian, k=1, which="LM", return_eigenvectors=False)[0]
    # 如果计算过程不收敛
    # 特征值的范围不超过2
    except Exception as e:
        largest_eigenvalues = 2
    
    # 调整后的对称归一化图拉普拉斯矩阵，L~ = 2 / Lambda * L - I
    rescaled_laplacian = (2./largest_eigenvalues)*laplacian - np.eye(laplacian.shape[0])
    
    # 生成k阶chebyshev不等式
    T_k = []
    T_k.append(np.eye(rescaled_laplacian.shape[0])) # T(0) = 1
    T_k.append(rescaled_laplacian)  # T(1) = x
    def chebyshev_recurrence(T_k_minus_one, T_k_minus_two, rescaled_laplacian):
        """
        定义切比雪夫递归公式  2 * x * T(k-1)(x) - T(k-2)(x), T(0) = 1, T(1) = x
        :param T_k_minus_one: T(k-1)(X)
        :param T_k_minus_two: T(k-2)(X)
        :param X: X
        :return: Tk(X)
        """
        # 递归公式：Tk(X) = 2X * T(k-1)(X) - T(k-2)(X)
        return 2 * rescaled_laplacian * T_k_minus_one - T_k_minus_two
    
    for i in range(2, MAX_DEGREE + 1):
        T_k.append(chebyshev_recurrence(T_k[-1], T_k[-2], rescaled_laplacian))
        
    support = MAX_DEGREE + 1
    graph = [X] + T_k
#     G = [keras.layers.Input(shape=(rescaled_laplacian.shape[0],)) for _ in range(support)]
#     G = [keras.layers.Input(shape=(None, None)) for _ in range(support)]
    
else:
    raise Exception('Invalid filter type.')

对角矩阵:
 [[0.4472136  0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.5        ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.5        0.         0.        ]
 [0.         0.         0.         ... 0.         0.5        0.        ]
 [0.         0.         0.         ... 0.         0.         0.57735027]]


In [31]:
len(graph), print([x.shape for x in graph])

[(2708, 1433), (2708, 2708), (2708, 2708), (2708, 2708)]


(4, None)

### 构建模型

In [32]:
T_k[0].shape

(2708, 2708)

In [69]:
# 定义基础的图卷积类
class GCN(keras.layers.Layer):
    def __init__(self,
                 units,
                 supports=1,        # 支持多图(input = [feature][adj[i]])
                 activation=None,   # 激活函数
                 use_bias=True,     # 如果存在偏置
                 kernel_initializer="glorot_uniform",
                 bias_initializer="zeros",
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.units = units  # 输出单元数
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias    # 输出是否增加偏置函数

        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)

        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)

        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.bias_constraint = keras.constraints.get(bias_constraint)

        self.supports_masking = True    # 是否支持masking操作
        self.supports = supports    # 多个图

        assert self.supports >= 1   # 图的个数大于1(即邻接矩阵个数大于1)


    def build(self, input_shape):
        """定义层中的参数
        Y = GraphConvolution(y.shape[1], support, activation='softmax')([H] + G)
        :param input_shape: 输入张量的形状
        :return:
        """
        print(input_shape)
        features_shape = tf.TensorShape(input_shape[0]).as_list()
        assert len(features_shape) == 2     # (batch_size, feature_dim)

        # 特征维度
        input_dim = features_shape[1]
        self.kernel = self.add_weight(shape=(input_dim * self.supports, self.units),
                                      initializer=self.kernel_initializer,
                                      name="kernel",
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name="bias",
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, inputs, mask=None):
        """编写层的功能逻辑
        Y = GraphConvolution(y.shape[1], support, activation='softmax')([H] + G)
        """
        # 输入的是隐层的feature和邻接矩阵list
        features = inputs[0]    # (2708, 1433)  => (node_num, feature_dim)
        A = inputs[1:]      # 对称归一化的邻接矩阵    [(2708, 2708), ...] => [(node_num, node_num), ...]

        # 多个图的情况
        supports = []
        for i in range(self.supports):
            supports.append(K.dot(A[i], features))   # (node_num, node_num) * (node_num, feature_dim) => (node_num, feature_dim)

        # 将多个图的结果拼接起来
        supports = K.concatenate(supports, axis=1)      # (node_num, feature_dim * self.supports)
        outputs = K.dot(supports, self.kernel)
        if self.use_bias:
            outputs = outputs + self.bias
        print("self.activation(outputs):", self.activation(outputs))
        return self.activation(outputs)

    def compute_output_shape(self, input_shape):
        return input_shape[0][0], self.units

In [70]:
X.shape, A.shape, y.shape

((2708, 1433), (2708, 2708), (2708, 7))

In [71]:
support

3

In [72]:
rescaled_laplacian.shape

(2708, 2708)

In [75]:
# 构建model
X_in = keras.layers.Input(shape=(X.shape[1],))
G = [keras.layers.Input(shape=(rescaled_laplacian.shape[0],)) for _ in range(support)]
x = X_in
H = keras.layers.Dropout(0.5)(x)
H = GCN(16, support, activation="relu", kernel_regularizer=keras.regularizers.l2(5e-4))([H] + G)
D = keras.layers.Dropout(0.5)(H)
D = keras.layers.Reshape((16,))(D)
Y = GCN(y.shape[1], support, activation="softmax")([D] + G)
Y = keras.layers.Reshape((7,))(Y)
# 编译模型
model = keras.models.Model(inputs=[X_in]+G,outputs=Y)

[TensorShape([Dimension(None), Dimension(1433)]), TensorShape([Dimension(None), Dimension(2708)]), TensorShape([Dimension(None), Dimension(2708)]), TensorShape([Dimension(None), Dimension(2708)])]
self.activation(outputs): Tensor("gcn_27/Relu:0", shape=(?, 16), dtype=float32)
[TensorShape([Dimension(None), Dimension(16)]), TensorShape([Dimension(None), Dimension(2708)]), TensorShape([Dimension(None), Dimension(2708)]), TensorShape([Dimension(None), Dimension(2708)])]
self.activation(outputs): Tensor("gcn_28/Softmax:0", shape=(?, 7), dtype=float32)


In [76]:
model.compile(loss='categorical_crossentropy', optimizer="sgd")

In [77]:
model.fit(graph, y_train,sample_weight=train_mask,
          batch_size=A.shape[0],epochs=350,shuffle=False,
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-2, patience=10,verbose=1)])

Epoch 1/350



tfdbg: caught SIGINT; calling sys.exit(1).


SystemExit: 1

/home/wangwei/miniconda3/envs/tf3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
